In [ ]:
# third party
import numpy as np

# syft absolute
import syft as sy

# Verify Version

In [ ]:
pip_info = !pip index versions syft
latest_deployed_version = pip_info[-1].split("LATEST: ")[-1].strip()

In [ ]:
# this notebook should only be used to run the latest deployed version of syft
# the notebooks after this (1a/1b and 2), will test migrating from that latest version
print(
    f"latest deployed version: {latest_deployed_version}, installed version: {sy.__version__}"
)
# assert (
#     latest_deployed_version == sy.__version__
# ), f"{latest_deployed_version} does not match installed version {sy.__version__}"

# Launch Server

In [ ]:
server = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    reset=True,
    n_consumers=2,
    create_producer=True,
    port="auto",
)

In [ ]:
client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
client.register(
    email="ds@openmined.org", name="John Doe", password="pw", password_verify="pw"
)

# Prepare some data to be migrated

In [ ]:
client_ds = server.login(email="ds@openmined.org", password="pw")

In [ ]:
dataset = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=np.array([10, 11, 12, 13, 14]),
            data=np.array([[15, 16, 17, 18, 19] for _ in range(100_000)]),
            mock_is_real=True,
        )
    ],
)

client.upload_dataset(dataset)

In [ ]:
data_low = client_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(datasite, data) -> float:
    # launch another job
    print("Computing mean...")
    return data

In [ ]:
req = client_ds.code.request_code_execution(compute_mean)

In [ ]:
client.requests[0].approve()

In [ ]:
job = client_ds.code.compute_mean(data=data_low, blocking=False)

In [ ]:
res = job.wait()

In [ ]:
res.get().shape

In [ ]:
# todo: add more data

In [ ]:
if server.server_type.value == "python":
    server.land()